# Knapsack Problem


En este notebook vamos a resolver el problema del "Knapsack Problem" utilizando un algoritmo de tensor networks.

En este problema se tiene un conjunto de objetos con un peso y un valor, y una mochila con una capacidad limitada. El objetivo es seleccionar los objetos que se pueden meter en la mochila de manera que se maximice el valor total, sin superar la capacidad de la mochila.


In [2]:
import numpy as np

from IPython.display import HTML, display



* Función global
* Función que genere los tensores
    * Función que determine qué tensores hay que crear
    * Función que cree cada tensor
* Función que contraiga los tensores


---
# Función que genera los tensores


In [3]:
def tensor_initial_generator(peso:int, valores:int, n_elementos:int, capacidad:int, weight_partial:int, tau:float=0)->np.array:
    """
    Funcion que genera el primer tensor de la cadena.
    Parameters
    ----------
    peso : int
        Peso del objeto.
    n_elementos : int
        Número de objetos disponibles.
    capacidad : int
        Capacidad de la mochila.
    position : int
        Posición del tensor en la lista.
    weight_partial : int
        Peso parcial de la mochila hasta este punto.

    Returns
    -------
    np.array
        Tensor inicial.
    """

    n_possible = min((capacidad - weight_partial)//peso, n_elementos) + 1
    max_weight_partial = min(capacidad + 1, (n_possible-1)*peso + weight_partial+1)

    # Inicializamos el tensor
    tensor = np.zeros((n_possible, max_weight_partial), dtype=float)

    # Rellenamos el tensor
    for n in range(n_possible):
        weight = weight_partial + peso*n
        tensor[n,weight] = np.exp(tau*(n-(n_possible-1)/2)*valores)

    return tensor

In [4]:
def tensor_intermediate_generator(peso:int, valores:int, n_elementos:int, capacidad:int, max_weight_partial:int, tau:float=0)->np.array:
    """
    Funcion que genera el tensor intermedio de la cadena.
    Parameters
    ----------
    peso : int
        Peso del objeto.
    n_elementos : int
        Número de objetos disponibles.
    capacidad : int
        Capacidad de la mochila.

    Returns
    -------
    np.array
        Tensor intermedio.
    """
    # Número de posibles elementos
    max_weight = min(capacidad+1, n_elementos*peso + max_weight_partial)

    # Inicializamos el tensor
    tensor = np.zeros((max_weight_partial, max_weight), dtype=float)

    # Rellenamos el tensor
    for weight_partial in range(max_weight_partial):
        n_possible = min((capacidad - weight_partial)//peso, n_elementos) + 1
        for n in range(n_possible):
            weight_down = weight_partial + peso*n
            tensor[weight_partial,weight_down] = np.exp(tau*n*valores)

    return tensor

In [5]:
def tensor_final_generator(peso:int, valores:int, n_elementos:int, capacidad:int, max_weight_partial:int, tau:float, extreme:bool=False, lambda_param:float=1)->np.array:
    """
    Funcion que genera el tensor final de la cadena.
    Parameters
    ----------
    peso : int
        Peso del objeto.
    n_elementos : int
        Número de objetos disponibles.
    capacidad : int
        Capacidad de la mochila.
    max_weight_partial : int
        Peso parcial máximo de la mochila hasta este punto.
    extreme:
        Si es True, solo permite el valor exacto final.

    Returns
    -------
    np.array
        Tensor final.
    """
    # Inicializamos el tensor
    tensor = np.zeros((max_weight_partial), dtype=float)

    # Rellenamos el tensor
    for weight_partial in range(max_weight_partial):
        n_possible = min((capacidad - weight_partial)//peso, n_elementos) + 1
        if extreme:
            weight_down = weight_partial + peso*(n_possible-1)
            if weight_down == capacidad:
                tensor[weight_partial] += np.exp(tau*(n_possible-1)*valores)
        else:
            for n in range(n_possible):
                weight_down = weight_partial + peso*n
                tensor[weight_partial] += np.exp(-tau*(lambda_param*(capacidad - weight_down)-n*valores))

    return tensor

Función que genera un tensor genérico.

In [6]:
def tensor_generator(pesos:np.array, valores:np.array, n_elementos:np.array, capacidad:int, tau:float, extreme:bool=False, lambda_param:float=1)->list:
    """
    Funcion que genera los tensores para la contraccion.
    Parameters
    ----------
    pesos : np.array
        Array de los pesos de cada uno de las clases de objetos disponibles.
    n_elementos : np.array
        Array del número de objetos disponibles para cada clase.
    capacidad : int
        Capacidad de la mochila.
    extreme:
        Si es True, solo permite el valor exacto final.

    Returns
    -------
    list
        Lista de los tensores a utilizar.
    """
    # Número de clases de objetos
    n_clases = len(pesos)

    # Lista de los tensores
    lista_tensores = []
    append = lista_tensores.append

    # Primer tensor
    tensor = tensor_initial_generator(pesos[0], valores[0], n_elementos[0], capacidad, 0, tau=tau)
    append(tensor)

    # Tensores intermedios
    for position in range(1, n_clases-1):
        tensor = tensor_intermediate_generator(pesos[position], valores[position], n_elementos[position], capacidad, tensor.shape[1], tau=tau)
        append(tensor)

    # Último tensor
    append(tensor_final_generator(pesos[-1], valores[-1], n_elementos[-1], capacidad, tensor.shape[1], tau, extreme=extreme, lambda_param=lambda_param))

    return lista_tensores

---
# Función de contracción

In [7]:
def contraction_tensors(lista_tensores:list):

    # Numero de tensores
    n_tensors = len(lista_tensores)
    vector = lista_tensores[-1]
    tensores_intermedios = [vector]
    append = tensores_intermedios.append

    for i in range(n_tensors-2, -1, -1):
        vector = lista_tensores[i] @ vector
        append(vector)

    tensores_intermedios.reverse()
    return vector, tensores_intermedios
    

---
# Funcion de peso parcial

In [8]:
def weight_solution(solution:np.array, pesos:np.array)->int:
    """
    Funcion que calcula el peso parcial de la mochila.
    Parameters
    ----------
    solution : np.array
        Solucion del problema.
    pesos : np.array
        Array de los pesos de cada uno de las clases de objetos disponibles.

    Returns
    -------
    int
        Peso parcial de la mochila.
    """
    return np.dot(solution, pesos)

def value_solution(solution:np.array, values:np.array)->int:
    """
    Funcion que calcula el peso parcial de la mochila.
    Parameters
    ----------
    solution : np.array
        Solucion del problema.
    pesos : np.array
        Array de los pesos de cada uno de las clases de objetos disponibles.

    Returns
    -------
    int
        Peso parcial de la mochila.
    """
    return np.dot(solution, values)

---
# Función global
	

Esta función es la que se encarga de resolver el problema del Knapsack utilizando un algoritmo de tensor networks.


In [9]:
def knapsack_solver(pesos:np.array, valores:np.array, n_elementos:np.array, capacidad:int, tau:float, extreme:bool=False, lambda_param:float=1)->np.array:
    """
    Funcion que resuelve el problema del Knapsack utilizando un algoritmo de tensor networks.
    Parameters
    ----------
    pesos : np.array
        Array de los pesos de cada uno de las clases de objetos disponibles.
    n_elementos : np.array
        Array del número de objetos disponibles para cada clase.
    capacidad : int
        Capacidad de la mochila.
    extreme:
        Si es True, solo permite el valor exacto final.

    Returns
    -------
    np.array
        Número de elementos escogidos para cada clase de los objetos seleccionados.
    """
    # Número de clases de objetos
    n_clases = len(pesos)

    # Array de la solucion
    solution = np.zeros(n_clases, dtype=int)

    if np.dot(n_elementos, pesos) <= capacidad:
        solution = np.array(n_elementos, dtype=int)
        return solution

    # Lista de los tensores a utilizar
    lista_tensores = tensor_generator(pesos, valores, n_elementos, capacidad, tau, extreme=extreme, lambda_param=lambda_param)
    for t in lista_tensores:
        print("t")
        print(t)
    # Paso inicial de la iteración
    vector_de_salida, tensores_intermedios = contraction_tensors(lista_tensores)

    # Obtenemos el valor correcto para esta variable
    solution[0] = np.argmax(vector_de_salida)

    # Proceso iterativo para el resto de variables
    for position in range(1, n_clases-1):
        # Peso de la solucion hasta este paso
        weight_partial = weight_solution(solution[:position], pesos[:position])
        if weight_partial == capacidad: break

        # Creamos el primer tensor de la cadena
        tensor_inicial = tensor_initial_generator(pesos[position], valores[position], n_elementos[position], capacidad, weight_partial, tau=tau)

        # Contraccion aprovechando los tensores intermedios
        vector_de_salida = tensor_inicial @ tensores_intermedios[position+1][:tensor_inicial.shape[1]]

        # Obtenemos el valor correcto para esta variable
        solution[position] = np.argmax(vector_de_salida)


    # Obtenemos el valor de la última variable
    weight_partial = weight_solution(solution[:-1], pesos[:-1])
    solution[-1] = max( min((capacidad - weight_partial)//pesos[-1], n_elementos[-1]), 0)

    return solution

---
# Prueba

In [10]:
def progress(value, max=100):
    return HTML("""
                <progress
                value='{value}'
                max='{max}'
                style='width: 50%'
                >
                {value}
                </progress>
                <p> Progreso: {porc} %</p>
                """.format(value=value, max=max, porc=np.round(value/max*100,2)))

In [15]:
n_clases = 10

capacidad = 23
np.random.seed(1)
pesos = np.random.randint(1,10, n_clases)
print(pesos)
valores = np.random.rand(n_clases)*1
#pesos = np.ones(n_clases, dtype=int)
n_elementos = np.array([3,]*n_clases) #np.random.randint(1,5, n_clases)

#tau = 1e1/(np.dot(valores, n_elementos) *capacidad/np.dot(pesos, n_elementos))
tau = 1
solution = knapsack_solver(pesos, valores, n_elementos, capacidad, tau, extreme=False, lambda_param=0)

#print('La solucion es: ', solution)
print('El peso de la solucion es:  ', weight_solution(solution, pesos))
print('El valor de la solucion es: ', value_solution(solution, valores))

[6 9 6 1 1 2 8 7 3 5]
t
[[0.53323568 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.81091077 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  1.23318131 0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  1.87534337]]
t
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         1.98420732 0.         0.
  0.         0.         0.         0.         0.         0.
  3.9370787  0.  

In [12]:
from ortools.algorithms.python import knapsack_solver as ks

new_pesos = []
new_valores = []
append1 = new_pesos.append
append2 = new_valores.append

for i in range(n_clases):
    for j in range(n_elementos[i]):
        append1(pesos[i])
        append2(int(valores[i]*1e6))

        
# Create the solver.
solver = ks.KnapsackSolver(
    ks.SolverType.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
    "KnapsackExample",
)

solver.init(new_valores, [new_pesos], [capacidad])
computed_value = solver.solve()

packed_items = []
packed_weights = []
total_weight = 0
print("Total value =", computed_value/1e6)
for i in range(len(new_valores)):
    if solver.best_solution_contains(i):
        packed_items.append(i)
        packed_weights.append(new_pesos[i])
        total_weight += new_pesos[i]
print("Total weight:", total_weight)
print("Packed items:", packed_items)
print("Packed_weights:", packed_weights)

Total value = 4.368463
Total weight: 23
Packed items: [3, 6, 7, 8, 10, 11, 15]
Packed_weights: [9, 1, 1, 1, 2, 2, 7]


In [13]:
n_casos = 100
'''
# - - - - - - - - - - - - - - - - - - - - 
n_clases = 100
capacidad = 1000
n_elementos = np.random.randint(1,10, n_clases)
# - - - - - - - - - - - - - - - - - - - - 
costes = np.zeros(n_casos, dtype=int)

out = display(progress(0, n_casos), display_id=True)
var_counter = 0

for i in range(n_casos):
    pesos = np.random.randint(1,20, n_clases)
    valores = np.random.rand(n_clases)*10
    solution = knapsack_solver(pesos, valores, n_elementos, capacidad, tau, extreme=True, lambda_param=5)
    costes[i] = abs(capacidad - weight_solution(solution, pesos))

    var_counter += 1
    out.update(progress(var_counter, n_casos))

print(sum(costes))
'''


'\n# - - - - - - - - - - - - - - - - - - - - \nn_clases = 100\ncapacidad = 1000\nn_elementos = np.random.randint(1,10, n_clases)\n# - - - - - - - - - - - - - - - - - - - - \ncostes = np.zeros(n_casos, dtype=int)\n\nout = display(progress(0, n_casos), display_id=True)\nvar_counter = 0\n\nfor i in range(n_casos):\n    pesos = np.random.randint(1,20, n_clases)\n    valores = np.random.rand(n_clases)*10\n    solution = knapsack_solver(pesos, valores, n_elementos, capacidad, tau, extreme=True, lambda_param=5)\n    costes[i] = abs(capacidad - weight_solution(solution, pesos))\n\n    var_counter += 1\n    out.update(progress(var_counter, n_casos))\n\nprint(sum(costes))\n'

# Comparativa

In [14]:
'''from time import time
time_total=time()
Q=100
elementos = 100
encontrado=False
cantidad = 12
np.random.seed(1)
#pesos = np.random.randint(1, 11, size=elementos)
print(len(pesos))
pesos.sort()
print(pesos)
Co= np.zeros((cantidad+1, 2))

for j in range(0,elementos):
    inicio=time()
    for i in range(0,Co.shape[0]):
        Co[i][0]=pesos[j]*(i%(cantidad+1))+Co[i][0]
        
        if Co[i][0]==Q:
            Co[i][j+1]=(i%(cantidad+1))
            L=Co[i]
            Co=0
            print(f"La fila con el valor más alto en la columna {0} es: {L}")
            encontrado=True
            break

        
        if Co[i][0]<Q:
            Co[i][j+1]=(i%(cantidad+1))
        else:
            Co[i][0]=-1
       
        if Co[i][0] != 0:
            for n in range(Co.shape[0]):
                if n != i and Co[i][0] == Co[n][0] and i>n:
                    Co[i][0]=-1

    if encontrado==True:
        break  
    Co = Co[Co[:, 0] != -1]
    
    print("cambio de objeto",j)
    fin=time()
    print(f"Tiempo de ejecución: {fin-inicio} segundos")
    #for i in range(0,Co.shape[0]):
        #print(np.round(Co[i]).astype(int))
    print(Co.shape[0])
    Co = np.repeat(Co, cantidad+1, axis=0)
    column_of_zeros = np.zeros((Co.shape[0], 1))  # Columna de ceros
    Co = np.hstack((Co, column_of_zeros))
    
indice_fila_max = np.argmax(Co[:, 0])

# Paso 2: Seleccionar la fila correspondiente al índice encontrado
fila_maxima = Co[indice_fila_max]

print(f"La fila con el valor más alto en la columna {0} es: {fila_maxima}")
fin_t=time()
print(f"Tiempo total de ejecución: {fin_t-time_total} segundos")'''

'from time import time\ntime_total=time()\nQ=100\nelementos = 100\nencontrado=False\ncantidad = 12\nnp.random.seed(1)\n#pesos = np.random.randint(1, 11, size=elementos)\nprint(len(pesos))\npesos.sort()\nprint(pesos)\nCo= np.zeros((cantidad+1, 2))\n\nfor j in range(0,elementos):\n    inicio=time()\n    for i in range(0,Co.shape[0]):\n        Co[i][0]=pesos[j]*(i%(cantidad+1))+Co[i][0]\n        \n        if Co[i][0]==Q:\n            Co[i][j+1]=(i%(cantidad+1))\n            L=Co[i]\n            Co=0\n            print(f"La fila con el valor más alto en la columna {0} es: {L}")\n            encontrado=True\n            break\n\n        \n        if Co[i][0]<Q:\n            Co[i][j+1]=(i%(cantidad+1))\n        else:\n            Co[i][0]=-1\n       \n        if Co[i][0] != 0:\n            for n in range(Co.shape[0]):\n                if n != i and Co[i][0] == Co[n][0] and i>n:\n                    Co[i][0]=-1\n\n    if encontrado==True:\n        break  \n    Co = Co[Co[:, 0] != -1]\n    \n 